In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import transformers
import pandas as pd
import torch
from tqdm import tqdm
from torchinfo import summary
from torch.utils.data import DataLoader
from transformers import AutoModel, AutoTokenizer
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [ ]:
model = AutoModel.from_pretrained("monologg/kobigbird-bert-base")
tokenizer = AutoTokenizer.from_pretrained("monologg/kobigbird-bert-base",
                                          additional_special_tokens=["[FACT]", "[ARGS]"]
                                          )
model.resize_token_embeddings(len(tokenizer))

Some weights of the model checkpoint at monologg/kobigbird-bert-base were not used when initializing BigBirdModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BigBirdModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tun

In [ ]:
print('모델에 적용된 스페셜 토큰 확인 : ', tokenizer.all_special_tokens)
print('모델에 적용된 스페셜 토큰 ids 확인 : ', tokenizer.all_special_ids)

모델에 적용된 스페셜 토큰 확인 :  ['<s>', '</s>', '[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]', '[FACT]', '[ARGS]']
모델에 적용된 스페셜 토큰 ids 확인 :  [5, 6, 1, 3, 0, 2, 4, 32500, 32501]


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/insurance_project/data/크롤링_데이터(전처리).xlsx')
df = df[df['label'] !=5] # 라벨 5 제외
df = df[['facts_pre', 'args_pre', 'label']]
df.head(2)

,facts_pre,args_pre,label
0,가. 보험계약의 체결 등 (1) 보험계약 체결 이 사건 보험계약자이자 신청인의 ...,가. 신청인의 주장 피보험자의 부주의로 인한 화재로 신청인 소유의 주택에 손해가 ...,0
1,가. 보험계약의 체결 등(1) 보험계약 체결 0000개발은 2021. 1. 20. ...,가. 신청인의 주장 사고가 덤프트럭을 후진하던 중 신호수(안전관리자)를 충격하여 발...,0


### config

In [ ]:
# 모든 설정 값
config = {
    'test_size' : 0.2,
    'batch_size' : 2,
    'num_workers' : 0,
    'epochs' : 1,
    'learning_rate' : 1e-5,
    'max_length' : 4096
}

In [ ]:
# train val split
X_train, X_test, y_train, y_test = train_test_split(df[['facts_pre', 'args_pre']], df['label'], test_size=config['test_size'], random_state=1)
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [ ]:
# dataset.py
class BigBirdDataset:
  def __init__(self, data, label):
    self.data = data
    self.label = label
    self.tokenizer = tokenizer
    self.max_len = config['max_length']

  def __len__(self):
    return len(self.data)

  def __getitem__(self, item):
    #  MAX_LENGTH가 FACTS 길이보다 짧으면 ARGS 토큰이 안나올수도 있다. (추가 개발 필요요)
    input = '[FACT]' + self.data['facts_pre'][item] + '[ARGS]' + self.data['args_pre'][item]
    label = self.label[item]
    inputs = self.tokenizer.encode_plus(
        input,
        truncation=True,
        max_length = self.max_len
    )

    ids = inputs["input_ids"]
    mask = inputs['attention_mask']
    token_type_ids = inputs["token_type_ids"]

    padding_length = self.max_len - len(ids)
    ids = ids + ([0] * padding_length)
    mask = mask + ([0] * padding_length)
    token_type_ids = token_type_ids + ([0] * padding_length)

    return {
        'ids' : torch.tensor(ids, dtype=torch.long),
        'attention_masks' : torch.tensor(mask, dtype=torch.long),
        'token_type_ids' : torch.tensor(token_type_ids, dtype=torch.long),
        'label' : torch.tensor(label, dtype=torch.float)
    }

In [ ]:
train_dataset = BigBirdDataset(X_train, y_train)
valid_dataset = BigBirdDataset(X_test, y_test)

In [ ]:
train_data_loader = DataLoader(train_dataset,
                               batch_size=config['batch_size'],
                               num_workers=config['num_workers'],)

valid_data_loader = DataLoader(valid_dataset,
                               batch_size=config['batch_size'],
                               num_workers=config['num_workers'])

### BigBirdClassifier

In [ ]:
class BigBirdClassifier(nn.Module):
    def __init__(self, model, num_classes):
        super().__init__()
        self.bigbird = model
        self.dropout = nn.Dropout(self.bigbird.config.hidden_dropout_prob)
        self.classifier = nn.Linear(self.bigbird.config.hidden_size, num_classes)  # 768 to 3

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bigbird(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        probabilities = F.softmax(logits, dim=-1)
        return probabilities, logits

In [ ]:
# 커스텀 모델 불러오기기
bigbirdModel = BigBirdClassifier(model, 3).to(device)

In [ ]:
# 손실함수
loss_fn = nn.CrossEntropyLoss()

# 옵티마이저
optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])

In [ ]:
len(train_dataset)

390

In [ ]:
len(train_data_loader)

195

In [ ]:
step_by_epoch = len(train_data_loader) // config['batch_size'] + 1
step_by_epoch

98

In [ ]:
for epoch in range(0, config['epochs']):
  print("")
  print('======== Epoch {:} / {:} ========'.format(epoch+1, config['epochs']))
  print('Training...')
  preds = []
  labels = []
  model.train()
  train_precision = 0.0
  train_recall = 0.0
  train_f1 = 0.0
  train_acc = 0.0
  train_loss = 0.0
  for batch_id, batch in enumerate(tqdm(train_data_loader)):
    # 배치 단위로 GPU로 로드드
    b_input_ids = batch['ids'].to(device)
    b_input_mask = batch['attention_masks'].to(device)
    b_token_type_id = batch['token_type_ids'].to(device)
    b_labels = batch['label'].to(device)

    # Forward 수행
    outputs, logits = bigbirdModel(b_input_ids,
                    token_type_ids=b_token_type_id,
                    attention_mask=b_input_mask
                    )

    # Accuracy 계산
    preds.extend(torch.argmax(outputs, axis=1).tolist())
    labels.extend(b_labels.tolist())
    ac = accuracy_score(labels, preds)
    train_acc += accuracy_score(labels, preds)

    # Precision, Recall, F1 score 계산
    train_precision += precision_score(labels, preds, average='macro')
    train_recall += recall_score(labels, preds, average='macro')
    train_f1 += f1_score(labels, preds, average='macro')

    # Loss 계산
    loss = loss_fn(logits, b_labels.long())
    train_loss += loss

    # Backward 수행 및 업데이트
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  print(f'epoch : {epoch+1}  Accuracy : {train_acc / len(train_data_loader):.4f}  Loss : {train_loss / len(train_data_loader):.4f}  Precision : {train_precision / len(train_data_loader):.4f}   Recall : {train_recall / len(train_data_loader):.4f}  F1-score : {train_f1 / len(train_data_loader):.4f}'  )


======== Epoch 1 / 1 ========
Training...


100%|██████████| 195/195 [00:10<00:00, 19.29it/s]

epoch : 1  Accuracy : 0.9338  Loss : 0.1457  Precision : 0.9335   Recall : 0.7851  F1-score : 0.8291


## 모델 저장

## 모델 평가

In [ ]:
from sklearn.metrics import classification_report

def evaluate(model, data_loader):
    model.eval()
    preds = []
    labels = []
    with torch.no_grad():
        for batch in tqdm(data_loader):
            b_input_ids = batch['ids'].to(device)
            b_input_mask = batch['attention_masks'].to(device)
            b_token_type_id = batch['token_type_ids'].to(device)
            b_labels = batch['label'].to(device)

            outputs, logits = model(b_input_ids,
                                    token_type_ids=b_token_type_id,
                                    attention_mask=b_input_mask)
            preds.extend(torch.argmax(outputs, axis=1).tolist())
            labels.extend(b_labels.tolist())

        # 정확도 계산
        accuracy = accuracy_score(labels, preds)

        # Precision, Recall, F1 score 계산
        report = classification_report(labels, preds, digits=4)
        print(report)
        return accuracy